In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option("display.max_row", None)
import numpy as np
import re
import nltk
import glob

In [4]:
# Function that calculates "points" for a card to be a "main entry" vs. "narrative," returns True if it's a "main entry"
def is_main_entry(index, row,df):
    # Check if previous and subsequent card authors match -> fix mispelled name in between
    if index > 0 and index < len(df)-1 and df.iloc[index-1]['Name'] == df.iloc[index+1]['Name'] and row['Name'] != df.iloc[index-1]['Name']:
        row['Name'] = df.iloc[index-1]['Name']
    # Check for new author -> automatic "main entry"
    elif index == 0 or row['Name'] != df.iloc[index-1]['Name']:
        return True
        
    # Calculate "points" for classification
    main_entry_pts, narrative_pts = 0, 0
    
    # Check length, longer -> narrative, shorter -> main entry
    if len(row['Text']) > 300:
        narrative_pts += 1
    elif len(row['Text']) < 200:
        main_entry_pts += 1
        
    # Check for misc. main entry identifiers ("GUIDE", "Section X", "X items") -> main entry
    if "Section" in row['Text']:
        main_entry_pts += 1
    if "items" in row['Text']:
        main_entry_pts += 1
    if "GUIDE" in row['Text'].upper() or "GUID" in row['Text'].upper() or "GUDE" in row['Text'].upper():
        main_entry_pts += 3
        
    # Check for misc. narrative identifiers
    if "FOR INFORMATION ABOUT THIS COLLECTION" in row['Text']:
        narrative_pts += 5
    if "Addition" in row['Text'] or "added" in row['Text']:
        narrative_pts += 1
    if not re.search(r"\d+", row['Text']):
        narrative_pts += 1
    if type(row['Text']) == str and type(df.iloc[index-1]['Text']) == str:
        # Matches begining of previous card
        if row['Text'][:50] == df.iloc[index-1]['Text'][:50]:
            narrative_pts += 1
        # Previous card indicates continuation of narrative onto current card
        if "(Continued on next card)" in df.iloc[index-1]['Text'] or "SEE NEXT CRD" in df.iloc[index-1]['Text'] or "Go to following card" in df.iloc[index-1]['Text']:
            narrative_pts += 5
        
    # Check for capitalized last name & card number at beginning of text (Card 2, JONES, Smith, John 3, etc.) -> narrative
    if re.search(r"Card [0-9]+", row['Text']):
        narrative_pts += 2
    elif (row['Name'].split(",")[0]).upper() in row['Text']:
        narrative_pts += 1
    elif re.search(r"{row['Name']} \d+", row['Text']):
        narrative_pts += 2
    
    # Tokenize card
    word_tokens = nltk.word_tokenize(row['Text'])
    tagged_text = nltk.pos_tag(word_tokens)
    
    # Check for "sentence format" (pos only present in narrative [verb, adj.]) -> narrative
    narr_pos = ["VB", "JJ", "PRP", "RB", "IN", "DT"]
    for word in tagged_text:
        if word[1] in narr_pos:
            narrative_pts += .3
    
    # Check for accessioned date (e.g., 13 MAR 97, 10-10-69) -> main entry
    if re.search(r"\d{1,2}-\d{1,2}-\d{2}", row['Text']):
        main_entry_pts += 1
    elif re.search(r"\d{1,2}\s[A-Z]{3}\s\d{2}", row['Text']):
        main_entry_pts += 2
    elif re.search(r"[A-Z]{3}\s\d{1,2}\s\d{4}", row['Text']):
        main_entry_pts += 2
        
#     print("{}: {} {}".format(index, main_entry_pts, narrative_pts))
        
    # Determine card classification
    if main_entry_pts >= narrative_pts:
        return True
    return False

In [5]:
def sort_collection(df):
    # Check every card and classify as "main entry" or "narrative", number by collection
    collection_num, collection_list = -1, []
    for index, row in df.iterrows():
        # Check for NaN values
        if pd.isna(row['Text']) or pd.isna(row['Name']) or row['Text'] == "" or row['Name'] == "":
            collection_list.append(-1)
        else:
            # Main entries signify a new collection
            if is_main_entry(index, row,df):
                collection_num += 1
            collection_list.append(collection_num)
    # Add column to dataframe with collections numbered
    df['Collection'] = collection_list
    # Label by collection headers as index, with original index
    df["Page_No_Within_Drawer"] = df.index
    curr = -100
    collection_header = []
    for index,row in df.iterrows():
        if(row["Collection"]!= curr):
            collection_header.append(row["Name"])
        else:
            collection_header.append("")
        curr = row["Collection"]
    df["Collection_Head"] = collection_header
    return df

In [6]:
# Sort all drawers and combine them
file_list_name = (glob.glob(r"catalog\file_name_checked/*"))
file_list = [pd.read_csv(i) for i in file_list_name]
sorted_list = [sort_collection(i) for i in file_list]
all_text = pd.concat(sorted_list)


In [8]:
all_text.to_csv("all_sorted_before_year_column.csv",index=False)

In [9]:
# add an is_collection_header simple column to label easily
li = []
n = -100
for i in all_text.Collection.values:
    if(n==-1):
        li.append(0)
    elif(n!=i):
        li.append(1)
    else:
        li.append(0)
    n=i
all_text["Is_collection_head"] = li


In [ ]:
# Create a year column for main entries
main_entries =  all_text[all_text["Is_collection_head"]==1]
ind_main_entries = all_text[all_text["Is_collection_head"]==1].index


# patterns used to extract sequentially if one does not exist
first ="(1\d{3}[-]\s?\d{2,4})"
second = "(1\d{3}.{0,9}1\d{0,3})"
third = "(1\d{3})"
fourth = "(1\d{3}[-]c(a)?\\.)"
fifth = "(n\\.\sd\\.)"    #no date

year_vals = main_entries.Text.str.extract(first)[0].fillna(  # if it returns na, find next pattern
    (main_entries.Text.str.extract(second)[0])).fillna((
        main_entries.Text.str.extract(third)[0])).fillna((
        main_entries.Text.str.extract(fourth)[0])).fillna((
        main_entries.Text.str.extract(fifth)[0])).values

all_text["Year"] =np.nan
all_text.iloc[ind_main_entries]["Year"]= year_vals
all_text.head(10)

In [ ]:
# run twice 
import import_ipynb
from upload_internet_archive import identifier_list
from split_to_drawers import real_size
drawer_no = ['157', '158', '159', '160', '161', '162', '163', '164', '165', '166', '167', '169', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '181', '182', '183', '184', '185', '186', '187', '188', '189', '190', '191', '193', '194', '195', '196', '197', '198', '199', '200', '201', '202', '203', '205', '206', '207', '208',
 '209', '210', '211', '212', '213', '214', '215', '217', '218', '219', '220', '221', '222', '223', '224', '225', '226', '227', '229', '230', '231']

In [25]:
# create link list for the dataset
link_identifier_list = ["https://archive.org/details/" + i  for i in identifier_list]
link_list = []
for i in range(len(link_identifier_list)):
    curr_size = real_size[i]
    curr_ident = link_identifier_list[i]
    # print(curr_size,curr_ident)
    for i in range(curr_size):
        link_list.append(curr_ident+"/page/n{}".format(i)) 


drawer_no_list = []
for i in range(len(real_size)):
    for j in range(real_size[i]):
        drawer_no_list.append(drawer_no[i])   
all_text["Link"] = link_list
all_text["Drawer_No"] = drawer_no_list

In [26]:
all_text.head()

,Name,Text,Collection,Page_No_Within_Drawer,Collection_Head,Is_collection_head,Year,Link,Drawer_No
0,A. B. Davis and Company,"A. B. Davis and Company (Philadelphia, Pa. ) See Davis (A. B.) and Company",0,0,A. B. Davis and Company,1,NaN,https://archive.org/details/rubensteinmanuscriptcatalog_A_to_Amer/page/n0,157
1,A.H. Roscoe,"AeHe Roscoe (Firm: Nashville, Tenne )e Journal, 1853, Septe-1857, Dece 1 item( 800 ppe )e Wholesale and retail druggist and dealer in paints, oils, and dyestuffse Summary: Journal (account book ) documents the sale of chemical and herbal drugs, paint and painting supplies, dyestuffs, personal and household supplies, and garden seeds to individuals and businessese le Drugstores--Tennesseee 2e Paint shops--Equipment and supplies-—- Tennesseee 3e Dyes and dyeinge 46 Household supplies--Tennesseee 5c Herbs--Therapeutic usee 6+ Seed industry and trade--Tennessees 7e Nashville ite® nne )—--Commercee 20 MAY QO1 23804905 NDHYme",1,1,A.H. Roscoe,1,NaN,https://archive.org/details/rubensteinmanuscriptcatalog_A_to_Amer/page/n1,157
2,Abbeville District,Abbeville District (8.C.) See South Carolina. Abbeville District,2,2,Abbeville District,1,NaN,https://archive.org/details/rubensteinmanuscriptcatalog_A_to_Amer/page/n2,157
3,"Abbott, William B","Abbott, William B. Papers, 1862-1864 Frederick Co., Va. Section A 5-16-57 GUIDE 10 items",3,3,"Abbott, William B",1,NaN,https://archive.org/details/rubensteinmanuscriptcatalog_A_to_Amer/page/n3,157
4,"Abbott, William B","Abbott, William B. Papers, 1862-1864, Fre- Gerick Co., Va. 10 items. Sketch These are the papers of William B. Abbott, evidently a well-to-do farmer of Frederick Co., Va. There are several documents concerned with the evaluation of damage done to his property by C. S. A. troops in 1862, and.receipts in 1864 for hay bought from Abbott at various times in Aug., 1864 by the C. S. A. Army.",3,4,,0,NaN,https://archive.org/details/rubensteinmanuscriptcatalog_A_to_Amer/page/n4,157


In [29]:
all_text.set_index("Collection_Head",inplace=True)
all_text.rename(columns = {"Page_No_Within_Drawer":"Page_drawer","Is_collection_head":"Coll_head"},inplace=True)
all_text = all_text[["Name","Text","Year","Page_drawer","Drawer_No","Link",'Coll_head',"Collection"]]

In [30]:
all_text.head()

,Name,Text,Year,Page_drawer,Drawer_No,Link,Coll_head,Collection
Collection_Head,,,,,,,,
A. B. Davis and Company,A. B. Davis and Company,"A. B. Davis and Company (Philadelphia, Pa. ) See Davis (A. B.) and Company",NaN,0,157,https://archive.org/details/rubensteinmanuscriptcatalog_A_to_Amer/page/n0,1,0
A.H. Roscoe,A.H. Roscoe,"AeHe Roscoe (Firm: Nashville, Tenne )e Journal, 1853, Septe-1857, Dece 1 item( 800 ppe )e Wholesale and retail druggist and dealer in paints, oils, and dyestuffse Summary: Journal (account book ) documents the sale of chemical and herbal drugs, paint and painting supplies, dyestuffs, personal and household supplies, and garden seeds to individuals and businessese le Drugstores--Tennesseee 2e Paint shops--Equipment and supplies-—- Tennesseee 3e Dyes and dyeinge 46 Household supplies--Tennesseee 5c Herbs--Therapeutic usee 6+ Seed industry and trade--Tennessees 7e Nashville ite® nne )—--Commercee 20 MAY QO1 23804905 NDHYme",NaN,1,157,https://archive.org/details/rubensteinmanuscriptcatalog_A_to_Amer/page/n1,1,1
Abbeville District,Abbeville District,Abbeville District (8.C.) See South Carolina. Abbeville District,NaN,2,157,https://archive.org/details/rubensteinmanuscriptcatalog_A_to_Amer/page/n2,1,2
"Abbott, William B","Abbott, William B","Abbott, William B. Papers, 1862-1864 Frederick Co., Va. Section A 5-16-57 GUIDE 10 items",NaN,3,157,https://archive.org/details/rubensteinmanuscriptcatalog_A_to_Amer/page/n3,1,3
,"Abbott, William B","Abbott, William B. Papers, 1862-1864, Fre- Gerick Co., Va. 10 items. Sketch These are the papers of William B. Abbott, evidently a well-to-do farmer of Frederick Co., Va. There are several documents concerned with the evaluation of damage done to his property by C. S. A. troops in 1862, and.receipts in 1864 for hay bought from Abbott at various times in Aug., 1864 by the C. S. A. Army.",NaN,4,157,https://archive.org/details/rubensteinmanuscriptcatalog_A_to_Amer/page/n4,0,3


In [31]:
all_text.to_csv("all_sorted_collection.csv")